In [ ]:
import sys
#!{sys.executable} -m pip install praw

In [ ]:
import sys
#!{sys.executable} -m pip install urllib

In [6]:
import praw
import pandas as pd
import datetime as dt
import tensorflow as tf

In [8]:
#connects to credential file 
from RedditCreds import client_id, client_secret, username, password



reddit = praw.Reddit(client_id=client_id, \
                     client_secret=client_secret, \
                     user_agent='Headliner', \
                     username=username, \
                     password=password)

In [9]:
#pick a reddit 

subreddit = reddit.subreddit('politics')



In [10]:
new_subreddit = subreddit.hot(limit=100000)


In [13]:
headlinelist=[]


In [14]:
for submission in new_subreddit:
    headlinelist.append(submission.title)
    


In [15]:

len(headlinelist)

863

In [ ]:
print(len(set(headlinelist)))
headlineset=set(headlinelist)
headlinestring=str(headlineset)
print(headlinestring.head(50))

In [17]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)

TensorFlow version: 2.3.1


In [20]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [21]:
#text=(topics_data.title.to_string(header=False, index=False))

In [22]:

text=headlinestring.lower()
#print(headlinestring[1115:1200])

 the people decide.', "Eric Trump claims his father received 'a vaccine' for Covid-19


In [23]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}


In [24]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [25]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [26]:
model = Sequential()
model.add(LSTM(700, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:

model.fit(X_modified, Y_modified, epochs=7, batch_size=50)

model.save_weights('text_generator_gigantic.h5')

Epoch 1/7
 401/1476 [=======>......................] - ETA: 1:47:17 - loss: 3.1412

In [ ]:
#model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_1003.h5')
model.load_weights('text_generator_gigantic.h5')

In [ ]:
#model.load_weights('text_generator_400_0.2_400_0.2_400_0.2_1003.h5')

In [ ]:
string_mapped = X[3333]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


In [ ]:
len(X)

In [ ]:
txt=""
for char in full_string:
    txt = txt+char
txt